In [ ]:
#complete code using Github link

def coppersmith_howgrave_univariate(pol, modulus, beta, mm, tt, XX):
    
    dd = pol.degree()
    nn = dd * mm + tt

    polZ = pol.change_ring(ZZ)
    x = polZ.parent().gen()

    # compute polynomials
    gg = []
    for ii in range(mm):
        for jj in range(dd):
            gg.append((x * XX)**jj * modulus**(mm - ii) * polZ(x * XX)**ii)
    for ii in range(tt):
        gg.append((x * XX)**ii * polZ(x * XX)**mm)

    # construct lattice B
    BB = Matrix(ZZ, nn)

    for ii in range(nn):
        for jj in range(ii+1):
            BB[ii, jj] = gg[ii][jj]

    # LLL
    BB = BB.LLL()

    # transform shortest vector in polynomial
    new_pol = 0
    for ii in range(nn):
        new_pol += x**ii * BB[0, ii] / XX**ii

    # factor polynomial
    potential_roots = new_pol.roots()
    
    # test roots
    roots = []
    for root in potential_roots:
        if root[0].is_integer():
            result = polZ(ZZ(root[0]))
            if gcd(modulus, result) >= modulus^beta:
                roots.append(ZZ(root[0]))

    return roots

e = 5
N = 84364443735725034864402554533826279174703893439763343343863260342756678609216895093779263028809246505955647572176682669445270008816481771701417554768871285020442403001649254405058303439906229201909599348669565697534331652019516409514800265887388539283381053937433496994442146419682027649079704982600857517093
C = 23701787746829110396789094907319830305538180376427283226295906585301889543996533410539381779684366880970896279018807100530176651625086988655210858554133345906272561027798171440923147960165094891980452757852685707020289384698322665347609905744582248157246932007978339129630067022987966706955482598869800151693

# RSA known parameters
ZmodN = Zmod(N);

def break_RSA(p_str, max_length_M):
    global e, C, ZmodN

    p_binary_str = ''.join(['{0:08b}'.format(ord(x)) for x in p_str])

    for length_M in range(0, max_length_M+1, 4):          # size of the root

        # Problem to equation (default)
        P.<M> = PolynomialRing(ZmodN) #, implementation='NTL')
        pol = ((int(p_binary_str, 2)<<length_M) + M)^e - C
        dd = pol.degree()

        # Tweak those
        beta = 1                                
        epsilon = beta / 7                      
        mm = ceil(beta**2 / (dd * epsilon))     
        tt = floor(dd * mm * ((1/beta) - 1))    
        XX = ceil(N**((beta**2/dd) - epsilon))  

        roots = coppersmith_howgrave_univariate(pol, N, beta, mm, tt, XX)

        if roots:
            print("Root is :", ' {0:b}'.format(roots[0]))
            return

    print('No solution found\n')

if __name__ == "__main__":
    break_RSA("You see a Gold-Bug in one corner. It is the key to a treasure found by ", 300)

In [7]:
#code to run using fpylll libraries to finally find LLL reduced matrix (..used sagemathcell)

from fpylll import IntegerMatrix,LLL
e = 5
N = 84364443735725034864402554533826279174703893439763343343863260342756678609216895093779263028809246505955647572176682669445270008816481771701417554768871285020442403001649254405058303439906229201909599348669565697534331652019516409514800265887388539283381053937433496994442146419682027649079704982600857517093
C = 23701787746829110396789094907319830305538180376427283226295906585301889543996533410539381779684366880970896279018807100530176651625086988655210858554133345906272561027798171440923147960165094891980452757852685707020289384698322665347609905744582248157246932007978339129630067022987966706955482598869800151693
a_str="You see a Gold-Bug in one corner. It is the key to a treasure found by "
K=2**64
a_binary_str = ''.join(['{0:08b}'.format(ord(x)) for x in a_str])
a_binary_str += '0'*64
a=int(a_binary_str, 2)
c0=a**5-C
c1=5*(a*a*a*a)*K
c2=10*(a*a*a)*K*K
c3=10*(a*a)*K*K*K
c4=5*a*(K**4)
c5=1*(K**5)
A=IntegerMatrix.from_matrix([[c5*(K**3), c4*(K**3), c3*(K**3), c2*(K**3), c1*(K**3), c0*(K**3),        0,    0,  0],
                             [        0,    c5*K*K,    c4*K*K,    c3*K*K,    c2*K*K,    c1*K*K,   c0*K*K,    0,  0],
                             [        0,         0,      c5*K,      c4*K,      c3*K,      c2*K,     c1*K, c0*K,  0],
                             [        0,         0,         0,        c5,        c4,        c3,       c2,   c1, c0],
                             [        0,         0,         0,         0,  N*(K**4),         0,        0,    0,  0],
                             [        0,         0,         0,         0,         0,  N*(K**3),        0,    0,  0],
                             [        0,         0,         0,         0,         0,         0, N*(K**2),    0,  0],
                             [        0,         0,         0,         0,         0,         0,        0,  N*K,  0],
                             [        0,         0,         0,         0,         0,         0,        0,    0,  N]])
L = LLL.reduction(A)
print(L)

ModuleNotFoundError: No module named 'fpylll'

In [3]:
#modified code in geeks for geeks for Newton Raphson method, have a look at it once.
#Link for the above (https://www.geeksforgeeks.org/program-for-newton-raphson-method/  )

# Python3 code for implementation of Newton Raphson Method for solving equations

#matrix from above LLL reduction
s = [[                                                                1088598874497692385189323969635398455789396411516641112845571288083476488567524885465577652947798879096950374330369383323519173438660269717493813390753202166709839161364030835577700000883346952699567419052420864231764845921598063583625740288,                                                                1640562279411734586407342543378115939236531345806107454916538279429895849205860146654259601188807950296123606460550340521124146969277645087802352657928546561653685500062381943944033662127690641822213223090419891623287981586644373298686722048,                                                                  241814201085691206758239930522383282088178796987724646409449187618241780387519675630291926902098419530452654718749162757663158266384107689906612626776437791893321336879615830871154652682922340304407919726874896332657362280097557437835378688,                                                                 -355259815743371327084932533841818574035173436528829261076925719253029829675324722217316777304398496078365333462026373886033874378054350451689664035751073726001334425828052366006852042111234764187534513783826777980157159289672179521524072448,                                                                   773180009293017106431413476791255633253106511371013081833989478960554365935037008696301595650647168264450051255337573279771091198921624324541559503764717823594724110310342341589953235046136637045889614274562498671082288384153800117265104896,                                                                  -1447772008682786192471092012339851064521821437409142464858347210384284743024606121210412777839500339204125172369053080051729949573107016529874964931707123710848806017749840402371196914789706694983719862901792321853893502512691837836474712064,                                                                    247468152724788251993744039366745669435913583261920082523343048370841770814889999184816713181341834318158233593180946547964221272650583830198063659351060802319390064486613352435439903925116262380353207887060576254983490635973753234171887616,                                                                   -2541215273830847591448248331833269006775180278254188515972793579860375751438647369083688538821439754609579319084475867571780186352681667246293710603117947902934063059355964674292845635052985504677887861357754197133377018828665511191448125440,                                                                     662896414833064830815631121764707897971391575441448548784637430311959892522509931178149876917794282577536052973400694379442596208531599462157236086253715033205328035885382823217826756756494531551850534650622078779919571065410662188743835550 ],
[                                                               -1503468679774105931193861419188910442951348861481321498859534083367403712660531377661764509214078637448167683357397816170823223178179006439169615228100919985309418394558723854947528679035121208276780270335674316436447809625228738024857141248,                                                                1965416080346952057125756667920680024819961483324409562215961581240007662875558042717464295847289811947553431883717129842537615396509963721799762329743667382164389522048547598842465886574823148821265633838083862950572074450349455750142623744,                                                                 2123047726251328413604618395278769282086665790866311581615699458194277307500545593810775582958361013469641069806036009038933383025402495755870840231136016994164606666563629788006728447588365942012087336842328468946102051772463765635425370112,                                                                   54801429002143206878174511718505975385373173958862506079617406788105148353898706889939160744862210200807530841538320561365935348842239984539878963610468523822988109596791842935208716679110792132556214550696373949694858090552404336686661632,                                                                 -1383125486826703353977441547560434406604909226717344983630487543169421114349608394577332554473924621746335407179839567801773574423275872261089060661409720465516228676962669009708433447802137311811423548651280914850279861343102277257500033024,                                                                    991089493524804399517507935101496108411209383434345821386613647107812419225027189677294378879995258771951881523132827551559481376359943046369968387341768334732816171653436873645635837871993245885107237953134566836456053456002104073721479168,                                                                    364512977941948304103078243568428898495896313174873190590080786676071647411538579289776295700187211708281307638506922313582538967986958094484280035758508543714455829555692156830370799652220627383915481822580988467896334832768790151487815680,                                                                   -1815382591309724022815677076166677284252014550972766463591878921761707972153524951735460249311296748901466171533269258108488451669832800771097514990636229089505214223763652757006941181280975412237266897131592136485303853028068697640299134976,                                                                433601057437984794534687254951482331808982063244917683087155612181950134762419088714990255774847528665021969997843133721651919771368917425731269769962302669318598901043666566452378718335709395999067079782462211374338314256930707734086949760 ],
[                                                                2584397459458598567967330540117047196410481670849578491291586511833848435682944033533679788328795581598214807610313039049066436600284378998587010150374803125057039274490442213743211507530994413799369509666060139936392688637396422297342967808,                                                                -147764022413277426210432116377545520706630210296848002965062405615447559915902937456009276853833897678571797347943392478601203908795603618464979336756615842031167454864801802091687221431984447600255353369412523773162911330447255791560818688,                                                                 -108690905690806145370918407067719114403755812472608269388668412051706745884357932209605765549056712761001988201897803745648622936094463807030467513292920032107048976927890453748968993729566899290381831595302077920992686395738440733500112896,                                                                 -484957679339374306196442314773448985593629555493421537095056181674237185174332870780167269717137551898838224370917887125143320741377128877824231466362916415469601985116568473501586217474852583196859951575021567643496174381481116232795029504,                                                                  -273562485015645048342489097982384269602768265253073971785748198298893250574800516614452595197035083799050723459975613716833964947214697588801830223971324118261598358597144439456001691739894799733839141984441283618400460538505467914218373120,                                                                   3612177734957286289010337760704027369734387147343922926734536660810300167943832312210650479626043694981682537496859530084504087413379596774737837283671515102796808506883426448031975803370997888078797572984751345342675800980604875945860399104,                                                                  -1320731903656031112192449354889865255772188438704655310873140298881425435707351852691870401057357600327022818094508954173833051155766673292068843152751706567137505936322230673760945983700038751566277673815474745373606883414846403016048771072,                                                                     972556947877741071328978001873277122450610022924531455033227909635651817885299910244761379802990083706901169735977907489173022176559164486580085110265041645901074982298540307967855077751009765561001369037400712698263285450743957913920864256,                                                                    -224064125162238688498391472441731623611822362581464678194259333851022625396839701381752007601581955041843238579300083649758660676245367990120118615132620311783712626867234822512555813652092407636901640282927783184590403669428987798572722235 ],
[                                                               -1653014567920942408696117791826258556669415012798710478423291303434007091706459953674550405841040810271392140110356385069207160391036684182933123239310854158793134716706615937268854294787766952761877159466077064940223836919429371462424723456,                                                                2886137390480221209212032729690238033122396603451009395866198931083257753850990904681998150688677881748171980729203069331369679497939499752486980329249802431560894509747266609609157085722628557529986273368841189717359949184631503677068148736,                                                                 -262473495940820757878876141368016544353814433329450719695913644698298239058581977591642019906335890599361599690389552994555964897084983915754528622112273191576393815036941410245868464311498331452200774467438208839557740590952250432192249856,                                                                 -576497233431897514219907130188441251417824416224532020689570941790289714582190184500331525689724822317574671580041207317834597252090657894623809209932156888272325452638789771427265397190716621796450742967631698803755788044372999870447878144,                                                                  -206484228736621420089736533870283979585594884083010128034050883899133187887471980526042629681813918368818636599713236869529400815017664891103660061001689167738132521523470454974877875280492802908205039973232906300750539222259158898112987136,                                                                  -1219176597735459131125471577291131648019780973587707082581622264050587489114227460106768566321082114473987670793658881878414860213051084687578480204706984299974384483973313552914475667335028813486204750953169854026683272145323796528074063872,                                                                   1206188488185856410218455563339771334480636359485659757730477434579735314033805743267816100370901622536260632661607317660619279261303018408765993210884315340142560683686365911016513738441716531298804860302085366308784449785032422341161779200,                                                                    2486439527740072761603391360669806228077579465438995202498627124612828155909495468739246871001109571880033987126831735673791766455495073667924985696346923119355472944670214209744337363942139171270869788445328955099962911543184303210981490688,                                                                    -701648376969201632551645871178030678800971611432453288666445661292814202352423872621248244912145427012089436653578639954152792704017728738424649308070988280797975847071129124409539602174924943898575255995217917830844867463049515296476692880 ],
[                                                                -957411135145835098527027932919601535875178218656048340084892843153677434406463582398484628870397241981073150173340329030644713584539130924676322280203485655265847237417923421232845766533971934753701558862748648186000790803285746108541698048,                                                                 663510906279817513546216773914436437907856980738305034463741600917732818127409309003405707993717729085789200632539844554504756753631550949406197429283045008854235404824773218478386255262660199225097514759267623895623364471544232919007494144,                                                                 2706021891661696371883975904587089273622283565152730499726253038308681335312235178996855355289398969030020446141808995251752107486796801453215798800731009289075187673458900495903206543394754044738795342349229311938717811279654253745007493120,                                                                  330611415844321032305934796257891661503104493355802719584777332986089727902321093130668137047544381545170158057672584240306450687577337628444484988161681184118469591579769587624524113848874324981645797042462380414485695870894606303816056832,                                                                  2800909886664678127992991921324204030293810018084739103920728335479201409911357333179096216684482905809009363744870800612852910109980259687895121342388960716888307146759774947469530978461884670474731365296179535032451968858187778881277132800,                                                                  -1159579253030496632689537053431878624160376272094961535724629075025068561915469635938669867070288045762248487187522400352139798678470628888081654561079104020362173323167601202626046493726755171264341989670422797670695593506597412916709818368,                                                                    667682035215250147023988381029828388593824731991484807623080550999649588083090653662085698087077522368004394064553358897563609910878186615353385552845275660631574878710389786258117693446651975549262619877487937799056690443459892718797324288,                                                                    1595412411189327418029712530374314104438693514963233016875236809496192695731850609042836900804011085498458874589912464068668040598447667234535067217240434582986881317879848051392964779291347188234866085132656539271006779633007437141929623552,                                                                    -451297431176815735006148587516042021117802419186049107676336739072002641661782038092635746705560785471319069971595366887042343198398662587046069874510921643021969341291461248940144090130092621306959480869725952336986788030392146524447373320 ],
[                                                                  56416268717276273389866446385256593639316140147820483796493986722101875425086454979018746043221456211593550395053132120023217390569872475446350230589472940226393977689114376694291444085613206768327547913516031951860127486336778377162129408,                                                                 238872704857859388835648690701354894441504528550505458221815500915804464473339547932250559463761366084321406631616448593436518446632280989883015658873385670091876672217458134801807282174374367877175000746801076311083464717971401698141274112,                                                                 -784792549680558225240349623121781041621588230689825013080541906204874710645780705488727129121321773782072846583514964845012445402597737856997871415877518392144613933789159515562217302491187937072618876873117695896449198781912997995781029888,                                                                -3448236820243748040152511409066448687010848869405138801875351856287661697850075776695835521831364611892344796135509370895273365813150214687422697828733163516794725262643069758297149700203869609921170845630534520572250035529479039632902455296,                                                                  2904054273752399964126794648751862824929016682934494196061452355134095679470630148891268587198486700644383470082643697835476491665919968813058878721740299617796777357619376564094657166888257857330295720519829097232632323347897981489752571904,                                                                    173895805953262432365657134379258446916970083748230909535235837753334013162434768597821428003942974491269146303422520317128906048559629939885410861429559267982814346152287916225389936774644537414266664169497902424143565165590481226740793344,                                                                  -1736524384255589558643907430406896719631303091245860221292760503074368402261093285614893177202507529051000482669585142365194506647079488940854505201451530205415887822351720246120238915618176931928443288359166905117223899443483984553079472128,                                                                     998912743326664091344354232645408981082589941431707273070042708263320517337045493131055059368516674704091601885196550182533994062403503219230466452495739294647606743550084198017105705065422271649383976850344693637691084764586160941478969344,                                                                    -154031862214507144293514688921456640881872516132488131430156153451567145304606024025595222727723914882185802582572296790309536361153361885308475407588121960885665687422739663623925809720573933904802422738433559348050072107964887451902034890 ],
[                                                               -2331062359441645886337268955532373730294584189990687632378615213715284354426096760665992937508262927041112581383868260342100193910705516364926724025114209136117838914711094875625182353655539330498042095181318198009321480824296200973257277440,                                                                1463677561308683538748099738932465511343532956183108618873622652274503353059265364372786191653394672668473217924471822085261817105883038566219029971163185849469890489536603434771886908202556937677813818054088962050621251014515095643441070080,                                                                -2504372285849159154941313487135597081194567359498532142179691357984594706941631574280110857094653711610056990891188333893254511480961759633125104276057439641574504469525408180005702987977970008992405532836008419234026989908940259016698757120,                                                                 1563051269014511282054905009790650655683107150920888211711698601373309236578273701680157693284679429558712386812084899159150675242347417354022996822114103100675739597642313548541431520362293825943771943465466175930381692292899330581340356608,                                                                  1051788324786340624351915629894173395798633001016179917945714156405230072571479840752611038068164748008759877433886762737305189445010663548411500477646679058250239467941972348493496069619376355796770914193606486646112233489821735014157516800,                                                                   1627979716114179122014259555587276849169719140494851714511616923443451151689149158159951390815069949672272217359754560664557130318113853587188398703612189298306325926396783258626772469045799026588423904128534715387228546850308993169477337088,                                                                  -3717140033777697440040920316400278533975222215643235485248543680799795971498685939159749689728381501830493578646155178820253763785554874109776756156207388555132560522705512558928998267302479674276394062472126047376572770784280193846023815168,                                                                    977263955494468206980475278583103145474888337526080482261823897030861112251778109797661168792090257573567908490653296570862101803990408579485495296316226110376994997562566220798952128864404466520529239520972494049765893119368793249221181440,                                                                     -38020282665826347558720961997968363065945961801397359796800245514262122725023738661961785187225523687945481030725773621616637817569602136492701747606390881343823505091197533470776139024423667869877294994481084585432316531959142622996187475 ],
[                                                               -2433875494926203578784561660774080556635314271595586278708647920324147478534966611876853380917526765350808228008726425196804298663619269427394712686165802489779388544334816285861495975289935496431877605245955742653043761365299519398011207680,                                                                 875457116810568322009211602920936828821859979968857124414311331458794121166980167227626257932993049130542645541772142186733665872338212402059304171575961207545494163479578004169592189664460006714512818260685964058260431301138413547251826688,                                                                -1388389379909709597899045912390608301099909669297459331811159609127896621075404125083025933202904317566684368861399748269379714156307264285461797694651624269303229296428138430701553484028726910811088792390285544802245841807932025354005250048,                                                                 3547016248636971407944408525897304664365915847822156354187081874271317457146137574205136238935702827348118236200168915839965037239645537216326064913243339886075796859212161062200027206847427259374915945112828352348400968379575266469839110144,                                                                  4687231515176479873703107047666769727742756660934669056824359673982861218430148617132894776812337916568012034221844514656811933397098956699384245935492935540642851889599452589286227227537861283372029430572757813738884611733676384314724450304,                                                                   2665662696179479282139500371464294013771834684577248371985819684434500807418666409640624484844833995563722885356675522106450991844451596651964871734565552067653095380491196735834358613455740672992469271257472106892913980277959007723688296448,                                                                   5425988821272359827661433249590369199738476432114637383840249796565549181345786125721493008799299630391881226334320986388931031038241518290662582915299055754268783654356745985325584605384915472560395002176375209150754729890267638158516551680,                                                                   -2571055325120400771281048748126987036003046017651884814721733315452659115155963416058781680305713371365629334663908299293971176747227070986830725163352527922962225339888622028792917177899883797320286973798609492086752510014704991597687209984,                                                                     231023081233440492180372680470603275903712241202461925639737219855439581357161792799667145552450149292612142892738067516751636893610280790654164740200729339069351650971793342475988051339389532063853943896987419448171983334244203212006548465 ],
[ 1583812091626984878217098714658772990701323131574558954919613261641705443030148020712513456319671212269214252283573086699625091442739392344768443985410061287906711381454424894640369279988340421396426291934278072887243464429623790405114806995814309562379429918609351071547869099028954896381953648940285952, 6119941748165695849220607751482803785287415834940601884213606909631041739703493180557072473443470776716537711111311557610174305702027746299715760863087265542250568476004196347185927220205126500072847258110400566770427383516264962236405077428823900767913558010217223881221034014596870177974623018307026944, 23647809736360053009978976916472341440292119084319200508958241166304731988423539549314737043284366701907405020431706695181731008989453458577461156560195145774350165902666544387815654468942473025177223606280928531571704678666319543611023607040061669474556948305975244389397477565002664641579435334540722176, 91376507871941367337208954599383434958491245913979227333842374311493052205588589784914908964371259174133793494883021432805785518010503093976724870753370555906190127170511348437393097264771674643674378147238994906707459196552800968974076735587241291517313734834626015275316153253862361675006239629933281280, 353084124236368715290425799302617012076067138677917717056483362971949435737331506248471637151386840753928669985223052176501844568271271972864895810725189769179673250365534364805731541294759268422734605612929746673210299593802839695712471060283090813460538165284027702244298987247113761052120340029608296448, 1364337527129824832322328598375312138272736416515663546914790493922322402641683858526998732633657251101621763608329323770930519823664830403192774271686105803883537804704797360610741025842609860135126911897101591011010162987067280928825774536510936586953196980118800695472309324323291992860934149722976813056, 5271879306271550661889679832520353217936710354943828409909821511248438399401913698317549077816494190926271106443989734575225388936523117029121686089524482254735987030738442070990140083387762689619016158577558984624312451814827223142626747422285337936116065272332160222434009656300940691353902054205269475328, 20370847291990938860997016650657402810973173945260045369883530088144160451611852656558519606830060347795367581580027886819625615123470333795543223415065495497710982844382170809058706205637021348974612269533123258848582596618419139099062950608389962959691177960369755589270499059008349863038587038694864060416, 78714134995456910721633168905719729498202825196761998532899039557817801146318810598541137536828808552046927841320708324052278793624511408744223476986287310805952903973619579971112713672445880770280615194682929841980853907064842269879728079594715514293476311718415427196558655426308067010795285871862613435633 ]]


def func( x, i=0):
    return s[i][0]*(x**8)+s[i][1]*(x**7)+s[i][2]*(x**6)+s[i][3]*(x**5)+s[i][4]*(x**4)+s[i][5]*(x**3)+s[i][6]*(x**2)+s[i][7]*(x)+s[i][8]

# Derivative of the above function
def derivFunc( x, i=0):
    return 8*s[i][0]*(x**7)+7*s[i][1]*(x**6)+6*s[i][2]*(x**5)+5*s[i][3]*(x**4)+4*s[i][4]*(x**3)+3*s[i][5]*(x**2)+2*s[i][6]*(x)+s[i][7]

# Function to find the root
def newtonRaphson( x, i=0):
    h = func(x, i) / derivFunc(x, i)
    while abs(h) >= 0.0000000000000001:
        h = func(x, i)/derivFunc(x, i)
        #x(i+1) = x(i) - f(x) / f'(x)
        x = x - h
    
    print("The value of the root is : ", int(x * 2**64)) #"%.4f"%

# Driver program to test above
x0 = 0.4 # Initial values assumed
for i in range(8):
    newtonRaphson(x0, i)

    
#below is the value computed using bahubali
#m = 4773930458381642785, note how close the values are:

The value of the root is :  4773930458381642752
The value of the root is :  4773930458381642752
The value of the root is :  4773930458381642752
The value of the root is :  4773930458381642752
The value of the root is :  4773930458381642752
The value of the root is :  9033472946001654784
The value of the root is :  4773930458381641728
The value of the root is :  4773930458381642752


In [9]:
#That's it Lower cells are not required, but code to find exact value of m is to be written. 
e = 5
N = 84364443735725034864402554533826279174703893439763343343863260342756678609216895093779263028809246505955647572176682669445270008816481771701417554768871285020442403001649254405058303439906229201909599348669565697534331652019516409514800265887388539283381053937433496994442146419682027649079704982600857517093
C = 23701787746829110396789094907319830305538180376427283226295906585301889543996533410539381779684366880970896279018807100530176651625086988655210858554133345906272561027798171440923147960165094891980452757852685707020289384698322665347609905744582248157246932007978339129630067022987966706955482598869800151693
a_str="You see a Gold-Bug in one corner. It is the key to a treasure found by "
a_binary_str = ''.join(['{0:08b}'.format(ord(x)) for x in a_str])
a_binary_str += '0'*64
a = int(a_binary_str, 2)

In [10]:
def polynomial(a, k, C):
    c0 = a**5 - C
    c1 = 5*(a*a*a*a)*K
    c2 = 10*(a*a*a)*K*K
    c3 = 10*(a*a)*K*K*K
    c4 = 5*a*(K**4)
    c5 = 1*(K**5)
    coef = [c0, c1, c2, c3, c4, c5]
    return coef

In [11]:
#interger matrix
A=IntegerMatrix.from_matrix([[c5*(K**3), c4*(K**3), c3*(K**3), c2*(K**3), c1*(K**3), c0*(K**3),        0,    0,  0],
                             [        0,    c5*K*K,    c4*K*K,    c3*K*K,    c2*K*K,    c1*K*K,   c0*K*K,    0,  0],
                             [        0,         0,      c5*K,      c4*K,      c3*K,      c2*K,     c1*K, c0*K,  0],
                             [        0,         0,         0,        c5,        c4,        c3,       c2,   c1, c0],
                             [        0,         0,         0,         0,  N*(K**4),         0,        0,    0,  0],
                             [        0,         0,         0,         0,         0,  N*(K**3),        0,    0,  0],
                             [        0,         0,         0,         0,         0,         0, N*(K**2),    0,  0],
                             [        0,         0,         0,         0,         0,         0,        0,  N*K,  0],
                             [        0,         0,         0,         0,         0,         0,        0,    0,  N]])

NameError: name 'IntegerMatrix' is not defined

In [6]:
# using join() + ord() + format()
# Converting String to binary
res = ''.join(format(ord(i), '08b') for i in test_str)

NameError: name 'test_str' is not defined